In [1]:
import numpy as np
np.set_printoptions(threshold=np.nan, linewidth=np.nan)

In [2]:
def read_hierarchy(filename):
    """
    Parses a hierarchy file of concepts. Subsumption is denoted by \t.
    Make sure that the hierarchy is not a tree. Tree is too nice in some properties.
    Currently the hierarchy is a forest with overlapping leaf nodes, making it maximally sparse by default.
    Returns:
        id_name: list of concepts as map: id -> concept
        name_id: map: concept -> id
        g: hierarchy graph as adjacency matrix
    """
    with open(filename, mode='r') as h:
        lines = h.read().split('\n')
    id_name = list(set(x.strip() for x in lines if x))
    n = len(id_name)  # number of concepts
    name_id = dict(zip(id_name, range(0, n)))
    g = np.zeros((n, n), dtype=np.bool)
    stack = list()
    for x in lines:
        depth = x.count('\t', 0, len(x) - len(x.lstrip()))
        name = x.strip()
        if depth < len(stack) - 1:  # arbitrary levels shallower
            del stack[depth:]  # pop until len(stack)==depth
        if depth == 0 and len(stack) == 0:  # root node
            stack.append(name)
        elif depth == len(stack):  # one level deeper
            from_id = name_id[stack[-1]]
            to_id = name_id[name]
            g[from_id, to_id] = True
            stack.append(name)
        elif depth == len(stack) - 1:  # same level
            from_id = name_id[stack[-2]]
            to_id = name_id[name]
            g[from_id, to_id] = True
            stack[-1] = name
        else:  # arbitrary levels shallower, but haven't reached root
            from_id = name_id[stack[-1]]
            to_id = name_id[name]
            g[from_id, to_id] = True
            stack.append(name)
    return id_name, name_id, g

In [3]:
id_name, name_id, H = read_hierarchy('hierarchy_full.txt')  # H for hierarchy (directed) subgraph
n = len(id_name)  # number of nodes. Currently 32, including 12 branch nodes and 20 leaf nodes

In [4]:
def print_bool_matrix(m):
    print(np.array_str(m.astype(np.uint8)))
def print_edges(g):  # defined outside: id_name
    edges = set()
    for x, y in np.argwhere(g):
        if g[y, x]:
            key1 = '{} <-> {}'.format(id_name[x], id_name[y])
            key2 = '{} <-> {}'.format(id_name[y], id_name[x])
            if key1 not in g and key2 not in g:
                edges.add(key1)
        else:
            edges.add('{} -> {}'.format(id_name[x], id_name[y]))
    print('\n'.join(edges))

In [5]:
from scipy.sparse.csgraph import floyd_warshall
H_ts = floyd_warshall(H, unweighted=True) != np.inf  # H_ts for transitive closure of H with self-loops
H_t = H_ts - np.eye(n, dtype=np.bool)  # H_t for transitive closure of H without self-loops

In [6]:
def build_exclusion(H_ts):
    """
    Builds exclusion subgraph from hierarchy. Two nodes are exclusive unless they share a common descendant.
    Args:
        H_ts: Transitive closure of hierarchical subgraph with self-loop. Self-loop is a must as the common descendant of two nodes may be one of them.
    """
    n = H_ts.shape[0]
    g = np.ones((n, n), dtype=np.bool) - np.eye(n, dtype=np.bool)  # fully connected by default
    for i in range(0, n):
        for j in range(i + 1, n):
            if np.logical_and(H_ts[i], H_ts[j]).any():  # if two nodes share a descendant, then unconnect
                g[i, j] = g[j, i] = False
    return g

In [7]:
EX = build_exclusion(H_ts)  # EX for exclusion (undirected) subgraph

In [8]:
def dfs_connected(g, from_id, to_id):
    def dfs(at_id):  # defined outside: visited
        if at_id == to_id:
            return True
        visited[at_id] = True
        for x in np.nonzero(g[at_id])[0]:  # nodes that can be visited from @at_id
            if not visited[x] and dfs(x):
                return True
        return False
    visited = np.zeros(n, dtype=np.bool)
    return dfs(from_id)

In [9]:
def sparsify(g, edges):
    """
    For each undirected edge (x, y), if both sides are stillconnected without (x, y), then (x, y) is removed.
    This algorithm does not guarantee maximal sparsity. See counterexample on Deng et al, page 8.
    Approximating |E| by |V|^2, this brute-force solution has complexity O(|V|^3).
    Args:
        edges: iterable<tuple<int, int>>
    """
    for x, y in edges:
        m = np.copy(g)
        m[x, y] = m[y, x] = False
        if dfs_connected(m, x, y) and dfs_connected(m, y, x):
            g[x, y] = g[y, x] = False
    return g

In [10]:
HEX_sparse = sparsify(H + EX, np.argwhere(EX))  # recall that @H is already maximally sparse
HEX_dense = H_t + EX - np.eye(n, n, dtype=np.bool)  # recall that @EX is already maximally dense

In [11]:
import pickle
with open('hex.pickle', mode='wb') as h:
    pickle.dump({'H': H, 'H_t': H_t, 'HEX_sparse': HEX_sparse, 'HEX_dense': HEX_dense}, h)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline
def visualize(m, directed):
    G = nx.DiGraph() if directed else nx.Graph()
    G.add_nodes_from(id_name)
    xs, ys = np.nonzero(m)
    xs = map(lambda x: id_name[x], xs)
    ys = map(lambda x: id_name[x], ys)
    G.add_edges_from(zip(xs, ys))
    # nx.write_dot(G, 'test.dot')
    plt.figure(figsize=(10, 8))
    nx.draw(G, pos=nx.spring_layout(G), node_color='white', node_size=2000, with_labels=True)
    plt.show()

In [ ]:
visualize(HEX_sparse, True)